# Column Transfer

In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('csv/covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [5]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [6]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(df.iloc[:,0:5],df.iloc[:,-1],test_size=0.3,random_state=0)
x_train

,age,gender,fever,cough,city
60,24,Female,102.0,Strong,Bangalore
80,14,Female,99.0,Mild,Mumbai
90,59,Female,99.0,Strong,Delhi
68,54,Female,104.0,Strong,Kolkata
51,11,Female,100.0,Strong,Kolkata
...,...,...,...,...,...
96,51,Female,101.0,Strong,Kolkata
67,65,Male,99.0,Mild,Bangalore
64,42,Male,104.0,Mild,Mumbai
47,18,Female,104.0,Mild,Bangalore


## Without Column Transfer

In [7]:
si=SimpleImputer()
x_train_fever=si.fit_transform(x_train[['fever']])
x_test_fever=si.transform(x_test[['fever']])

In [8]:
oe=OrdinalEncoder(categories=[['Mild','Strong']])
x_train_cough=oe.fit_transform(x_train[['cough']])
x_test_cough=oe.transform(x_test[['cough']])

In [9]:
ohe=OneHotEncoder(drop='first',sparse_output=False)
x_train_gender_city=ohe.fit_transform(x_train[['gender','city']])
x_test_gender_city=ohe.transform(x_test[['gender','city']])

In [10]:
x_train_age=x_train.drop(['gender','city','cough','fever'],axis=1).values
x_test_age=x_test.drop(['gender','city','cough','fever'],axis=1).values

In [11]:
x_train_transformed=np.concatenate([x_train_age,x_train_gender_city,x_train_cough,x_train_fever],axis=1)
x_test_transformed=np.concatenate([x_test_age,x_test_gender_city,x_test_cough,x_test_fever],axis=1)
x_train_transform=pd.DataFrame(x_train_transformed)
x_test_transform=pd.DataFrame(x_test_transformed)
x_train_transform

,0,1,2,3,4,5,6
0,24.0,0.0,0.0,0.0,0.0,1.0,102.0
1,14.0,0.0,0.0,0.0,1.0,0.0,99.0
2,59.0,0.0,1.0,0.0,0.0,1.0,99.0
3,54.0,0.0,0.0,1.0,0.0,1.0,104.0
4,11.0,0.0,0.0,1.0,0.0,1.0,100.0
...,...,...,...,...,...,...,...
65,51.0,0.0,0.0,1.0,0.0,1.0,101.0
66,65.0,1.0,0.0,0.0,0.0,0.0,99.0
67,42.0,1.0,0.0,0.0,1.0,0.0,104.0
68,18.0,0.0,0.0,0.0,0.0,0.0,104.0


## With Column Transform

In [12]:
from sklearn.compose import ColumnTransformer

In [16]:
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(drop='first', sparse_output=False), ['gender', 'city'])
], remainder='passthrough')

x_train_transformed = transformer.fit_transform(x_train)
x_test_transformed = transformer.transform(x_test)
x_train_transformed

array([[102.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  24.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  14.        ],
       [ 99.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  59.        ],
       [104.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  54.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  11.        ],
       [102.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  33.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  64.        ],
       [101.06557377,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  71.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          0.    

In [14]:
x_train_transformed=pd.DataFrame(x_train_transformed)
x_test_transformed=pd.DataFrame(x_test_transformed)
x_train_transformed.head()

,0,1,2,3,4,5,6
0,102.0,1.0,0.0,0.0,0.0,0.0,24.0
1,99.0,0.0,0.0,0.0,0.0,1.0,14.0
2,99.0,1.0,0.0,1.0,0.0,0.0,59.0
3,104.0,1.0,0.0,0.0,1.0,0.0,54.0
4,100.0,1.0,0.0,0.0,1.0,0.0,11.0
